In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [8]:
class Arg:
    arch = 'resnet18'
    data = '/data/imagenet/ILSVRC/Data/CLS-LOC/'
    batch_size = 128
    workers = 4
    start_epoch = 0
    epochs = 90
    lr = 0.1
    momentum = 0.9
    weight_decay = 1e-4
    print_freq = 10
    resume = ''
    world_size = -1
    rank = -1
    dist_url = 'tcp://224.66.41.62:23456'
    seed = None
    dist_backend = 'nccl'
    gpu = None
    distributed = False
    pretrained = False
    evaluate = False
    multiprocessing_distributed = False
args = Arg()

In [3]:
best_acc1 = 0

In [4]:
ngpus_per_node = torch.cuda.device_count()

In [5]:
def train(train_loader, model, criterion, optimizer, epoch, args):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            images = images.cuda(args.gpu, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(args.gpu, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i)


def validate(val_loader, model, criterion, args):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if args.gpu is not None:
                images = images.cuda(args.gpu, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(args.gpu, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [6]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [7]:
def adjust_learning_rate(optimizer, epoch, args):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [11]:
def main_worker(gpu, ngpus_per_node, args):
    global best_acc1
    args.gpu = gpu

    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))

    if args.distributed:
        if args.dist_url == "env://" and args.rank == -1:
            args.rank = int(os.environ["RANK"])
        if args.multiprocessing_distributed:
            # For multiprocessing distributed training, rank needs to be the
            # global rank among all the processes
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                world_size=args.world_size, rank=args.rank)
    # create model
    if args.pretrained:
        print("=> using pre-trained model '{}'".format(args.arch))
        model = models.__dict__[args.arch](pretrained=True)
    else:
        print("=> creating model '{}'".format(args.arch))
        model = models.__dict__[args.arch]()

    if not torch.cuda.is_available():
        print('using CPU, this will be slow')
    elif args.distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            model.cuda(args.gpu)
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int((args.workers + ngpus_per_node - 1) / ngpus_per_node)
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        else:
            model.cuda()
            # DistributedDataParallel will divide and allocate batch_size to all
            # available GPUs if device_ids are not set
            model = torch.nn.parallel.DistributedDataParallel(model)
    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        model = model.cuda(args.gpu)
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        if args.arch.startswith('alexnet') or args.arch.startswith('vgg'):
            model.features = torch.nn.DataParallel(model.features)
            model.cuda()
        else:
            model = torch.nn.DataParallel(model).cuda()

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda(args.gpu)

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            if args.gpu is None:
                checkpoint = torch.load(args.resume)
            else:
                # Map model to be loaded to specified single gpu.
                loc = 'cuda:{}'.format(args.gpu)
                checkpoint = torch.load(args.resume, map_location=loc)
            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_acc1']
            if args.gpu is not None:
                # best_acc1 may be from a checkpoint from a different GPU
                best_acc1 = best_acc1.to(args.gpu)
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    traindir = '/data/imagenet_augment_MotionBlur/motion_blur/3'
    valdir = os.path.join(args.data, 'val')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_dataset = datasets.ImageFolder(
        traindir,
        transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None),
        num_workers=args.workers, pin_memory=True, sampler=train_sampler)

    val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True)

    if args.evaluate:
        validate(val_loader, model, criterion, args)
        return

    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            train_sampler.set_epoch(epoch)
        adjust_learning_rate(optimizer, epoch, args)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, args)

        # evaluate on validation set
        acc1 = validate(val_loader, model, criterion, args)

        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        if not args.multiprocessing_distributed or (args.multiprocessing_distributed
                and args.rank % ngpus_per_node == 0):
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': args.arch,
                'state_dict': model.state_dict(),
                'best_acc1': best_acc1,
                'optimizer' : optimizer.state_dict(),
            }, is_best)

In [12]:
main_worker(args.gpu, ngpus_per_node, args)

=> creating model 'resnet18'
Epoch: [0][   0/1002]	Time  1.109 ( 1.109)	Data  0.856 ( 0.856)	Loss 7.1297e+00 (7.1297e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Epoch: [0][  10/1002]	Time  0.320 ( 0.399)	Data  0.000 ( 0.098)	Loss 7.3218e+00 (7.1510e+00)	Acc@1   0.00 (  0.07)	Acc@5   0.78 (  0.64)
Epoch: [0][  20/1002]	Time  0.324 ( 0.365)	Data  0.000 ( 0.062)	Loss 7.2023e+00 (7.2196e+00)	Acc@1   0.00 (  0.11)	Acc@5   1.56 (  0.71)
Epoch: [0][  30/1002]	Time  0.329 ( 0.356)	Data  0.000 ( 0.049)	Loss 6.9936e+00 (7.1881e+00)	Acc@1   0.00 (  0.10)	Acc@5   1.56 (  0.83)
Epoch: [0][  40/1002]	Time  0.325 ( 0.350)	Data  0.000 ( 0.043)	Loss 7.0098e+00 (7.1483e+00)	Acc@1   0.78 (  0.17)	Acc@5   1.56 (  0.80)
Epoch: [0][  50/1002]	Time  0.326 ( 0.345)	Data  0.000 ( 0.039)	Loss 7.0442e+00 (7.1164e+00)	Acc@1   0.00 (  0.17)	Acc@5   0.00 (  0.77)
Epoch: [0][  60/1002]	Time  0.326 ( 0.342)	Data  0.000 ( 0.036)	Loss 6.9403e+00 (7.0906e+00)	Acc@1   0.78 (  0.15)	Acc@5   1.56 (  0.83)
Epoch: [0][ 

Epoch: [0][ 600/1002]	Time  0.329 ( 0.331)	Data  0.000 ( 0.024)	Loss 6.4649e+00 (6.7302e+00)	Acc@1   0.78 (  0.52)	Acc@5   3.12 (  2.15)
Epoch: [0][ 610/1002]	Time  0.331 ( 0.331)	Data  0.000 ( 0.024)	Loss 6.5256e+00 (6.7268e+00)	Acc@1   0.00 (  0.52)	Acc@5   2.34 (  2.16)
Epoch: [0][ 620/1002]	Time  0.333 ( 0.331)	Data  0.000 ( 0.024)	Loss 6.5020e+00 (6.7233e+00)	Acc@1   0.78 (  0.53)	Acc@5   3.12 (  2.18)
Epoch: [0][ 630/1002]	Time  0.327 ( 0.331)	Data  0.000 ( 0.024)	Loss 6.5817e+00 (6.7204e+00)	Acc@1   0.78 (  0.54)	Acc@5   3.91 (  2.20)
Epoch: [0][ 640/1002]	Time  0.331 ( 0.331)	Data  0.000 ( 0.024)	Loss 6.4212e+00 (6.7173e+00)	Acc@1   0.00 (  0.54)	Acc@5   4.69 (  2.21)
Epoch: [0][ 650/1002]	Time  0.331 ( 0.331)	Data  0.000 ( 0.024)	Loss 6.4261e+00 (6.7146e+00)	Acc@1   0.78 (  0.55)	Acc@5   3.91 (  2.23)
Epoch: [0][ 660/1002]	Time  0.332 ( 0.331)	Data  0.000 ( 0.024)	Loss 6.5963e+00 (6.7120e+00)	Acc@1   0.78 (  0.55)	Acc@5   3.91 (  2.26)
Epoch: [0][ 670/1002]	Time  0.332 ( 0.331

Test: [240/391]	Time  0.678 ( 0.357)	Loss 6.0325e+00 (6.3817e+00)	Acc@1   8.59 (  1.40)	Acc@5  17.97 (  5.06)
Test: [250/391]	Time  0.101 ( 0.356)	Loss 6.1088e+00 (6.3765e+00)	Acc@1   0.00 (  1.38)	Acc@5   7.03 (  5.07)
Test: [260/391]	Time  0.923 ( 0.357)	Loss 6.5065e+00 (6.3773e+00)	Acc@1   0.00 (  1.38)	Acc@5   1.56 (  5.06)
Test: [270/391]	Time  0.144 ( 0.354)	Loss 6.2342e+00 (6.3812e+00)	Acc@1   0.00 (  1.34)	Acc@5   2.34 (  4.95)
Test: [280/391]	Time  1.065 ( 0.356)	Loss 6.6405e+00 (6.3835e+00)	Acc@1   0.00 (  1.32)	Acc@5   0.00 (  4.93)
Test: [290/391]	Time  0.525 ( 0.354)	Loss 6.5773e+00 (6.3892e+00)	Acc@1   0.00 (  1.27)	Acc@5   0.00 (  4.80)
Test: [300/391]	Time  0.100 ( 0.353)	Loss 6.5320e+00 (6.3942e+00)	Acc@1   0.00 (  1.23)	Acc@5   0.00 (  4.70)
Test: [310/391]	Time  0.823 ( 0.353)	Loss 6.3311e+00 (6.3993e+00)	Acc@1  14.06 (  1.28)	Acc@5  22.66 (  4.71)
Test: [320/391]	Time  0.472 ( 0.353)	Loss 6.0299e+00 (6.4021e+00)	Acc@1   0.00 (  1.25)	Acc@5   2.34 (  4.64)
Test: [330

Epoch: [1][ 470/1002]	Time  0.347 ( 0.335)	Data  0.000 ( 0.024)	Loss 6.1349e+00 (6.2665e+00)	Acc@1   3.12 (  1.75)	Acc@5   8.59 (  5.91)
Epoch: [1][ 480/1002]	Time  0.346 ( 0.336)	Data  0.000 ( 0.024)	Loss 6.1710e+00 (6.2651e+00)	Acc@1   2.34 (  1.75)	Acc@5   7.81 (  5.92)
Epoch: [1][ 490/1002]	Time  0.347 ( 0.336)	Data  0.000 ( 0.024)	Loss 6.1020e+00 (6.2626e+00)	Acc@1   3.12 (  1.77)	Acc@5   7.81 (  5.96)
Epoch: [1][ 500/1002]	Time  0.342 ( 0.336)	Data  0.000 ( 0.024)	Loss 6.3812e+00 (6.2620e+00)	Acc@1   0.78 (  1.77)	Acc@5   3.91 (  5.95)
Epoch: [1][ 510/1002]	Time  0.336 ( 0.336)	Data  0.000 ( 0.024)	Loss 6.1301e+00 (6.2603e+00)	Acc@1   1.56 (  1.79)	Acc@5   7.03 (  5.98)
Epoch: [1][ 520/1002]	Time  0.338 ( 0.336)	Data  0.000 ( 0.024)	Loss 6.1444e+00 (6.2577e+00)	Acc@1   1.56 (  1.80)	Acc@5   7.81 (  6.02)
Epoch: [1][ 530/1002]	Time  0.331 ( 0.336)	Data  0.000 ( 0.024)	Loss 6.1311e+00 (6.2560e+00)	Acc@1   1.56 (  1.80)	Acc@5   8.59 (  6.03)
Epoch: [1][ 540/1002]	Time  0.330 ( 0.336

Test: [ 80/391]	Time  0.103 ( 0.359)	Loss 6.2313e+00 (6.2272e+00)	Acc@1   0.00 (  2.05)	Acc@5   0.00 (  7.46)
Test: [ 90/391]	Time  0.103 ( 0.363)	Loss 6.4722e+00 (6.2755e+00)	Acc@1   0.00 (  1.88)	Acc@5   2.34 (  7.08)
Test: [100/391]	Time  0.103 ( 0.349)	Loss 6.5462e+00 (6.3145e+00)	Acc@1   1.56 (  1.73)	Acc@5   6.25 (  6.68)
Test: [110/391]	Time  0.104 ( 0.345)	Loss 6.8326e+00 (6.3710e+00)	Acc@1   0.00 (  1.61)	Acc@5   0.00 (  6.25)
Test: [120/391]	Time  0.682 ( 0.351)	Loss 6.1288e+00 (6.3576e+00)	Acc@1   0.00 (  1.51)	Acc@5   3.91 (  6.30)
Test: [130/391]	Time  0.100 ( 0.346)	Loss 6.6365e+00 (6.3202e+00)	Acc@1   0.00 (  1.68)	Acc@5   0.78 (  6.61)
Test: [140/391]	Time  0.581 ( 0.341)	Loss 7.2105e+00 (6.3491e+00)	Acc@1   0.00 (  1.61)	Acc@5   0.78 (  6.28)
Test: [150/391]	Time  0.767 ( 0.341)	Loss 6.0842e+00 (6.3553e+00)	Acc@1   6.25 (  1.65)	Acc@5  14.84 (  6.29)
Test: [160/391]	Time  0.103 ( 0.335)	Loss 6.0991e+00 (6.3141e+00)	Acc@1   0.78 (  1.88)	Acc@5   2.34 (  6.70)
Test: [170

...

Test: [ 30/391]	Time  0.105 ( 0.378)	Loss 6.6202e+00 (5.3408e+00)	Acc@1  10.16 ( 17.72)	Acc@5  19.53 ( 36.72)
Test: [ 40/391]	Time  0.122 ( 0.357)	Loss 6.7296e+00 (5.1108e+00)	Acc@1  10.16 ( 20.27)	Acc@5  22.66 ( 39.18)
Test: [ 50/391]	Time  0.105 ( 0.361)	Loss 3.6908e+00 (5.2700e+00)	Acc@1  25.78 ( 19.78)	Acc@5  50.78 ( 37.65)
Test: [ 60/391]	Time  0.102 ( 0.346)	Loss 5.4242e+00 (5.1496e+00)	Acc@1  14.84 ( 20.76)	Acc@5  34.38 ( 39.04)
Test: [ 70/391]	Time  0.105 ( 0.337)	Loss 6.2821e+00 (5.2544e+00)	Acc@1   7.81 ( 19.53)	Acc@5  17.19 ( 37.49)
Test: [ 80/391]	Time  0.123 ( 0.339)	Loss 5.7038e+00 (5.3650e+00)	Acc@1  11.72 ( 18.27)	Acc@5  28.91 ( 36.07)
Test: [ 90/391]	Time  0.105 ( 0.331)	Loss 5.0679e+00 (5.3848e+00)	Acc@1  16.41 ( 17.68)	Acc@5  35.94 ( 35.53)
Test: [100/391]	Time  0.104 ( 0.332)	Loss 5.2659e+00 (5.3693e+00)	Acc@1   9.38 ( 17.42)	Acc@5  33.59 ( 35.51)
Test: [110/391]	Time  0.102 ( 0.327)	Loss 5.7294e+00 (5.4221e+00)	Acc@1  11.72 ( 16.96)	Acc@5  33.59 ( 34.98)
Test: [120

Epoch: [88][ 300/1002]	Time  0.336 ( 0.330)	Data  0.000 ( 0.025)	Loss 1.3416e+00 (1.2427e+00)	Acc@1  74.22 ( 72.91)	Acc@5  83.59 ( 87.71)
Epoch: [88][ 310/1002]	Time  0.333 ( 0.330)	Data  0.000 ( 0.025)	Loss 1.5160e+00 (1.2452e+00)	Acc@1  71.09 ( 72.88)	Acc@5  84.38 ( 87.67)
Epoch: [88][ 320/1002]	Time  0.328 ( 0.330)	Data  0.000 ( 0.025)	Loss 1.2074e+00 (1.2463e+00)	Acc@1  74.22 ( 72.87)	Acc@5  89.06 ( 87.63)
Epoch: [88][ 330/1002]	Time  0.335 ( 0.330)	Data  0.000 ( 0.024)	Loss 1.6577e+00 (1.2475e+00)	Acc@1  66.41 ( 72.84)	Acc@5  77.34 ( 87.59)
Epoch: [88][ 340/1002]	Time  0.331 ( 0.331)	Data  0.000 ( 0.024)	Loss 1.4255e+00 (1.2460e+00)	Acc@1  66.41 ( 72.87)	Acc@5  87.50 ( 87.63)
Epoch: [88][ 350/1002]	Time  0.333 ( 0.331)	Data  0.000 ( 0.024)	Loss 1.0393e+00 (1.2453e+00)	Acc@1  74.22 ( 72.88)	Acc@5  91.41 ( 87.64)
Epoch: [88][ 360/1002]	Time  0.327 ( 0.331)	Data  0.000 ( 0.024)	Loss 1.1531e+00 (1.2455e+00)	Acc@1  75.00 ( 72.89)	Acc@5  88.28 ( 87.65)
Epoch: [88][ 370/1002]	Time  0.330

Epoch: [88][ 900/1002]	Time  0.325 ( 0.334)	Data  0.000 ( 0.024)	Loss 1.4026e+00 (1.2527e+00)	Acc@1  67.19 ( 72.68)	Acc@5  83.59 ( 87.56)
Epoch: [88][ 910/1002]	Time  0.326 ( 0.334)	Data  0.000 ( 0.024)	Loss 1.3157e+00 (1.2525e+00)	Acc@1  70.31 ( 72.69)	Acc@5  83.59 ( 87.57)
Epoch: [88][ 920/1002]	Time  0.325 ( 0.334)	Data  0.000 ( 0.024)	Loss 1.0926e+00 (1.2526e+00)	Acc@1  77.34 ( 72.68)	Acc@5  90.62 ( 87.56)
Epoch: [88][ 930/1002]	Time  0.326 ( 0.334)	Data  0.000 ( 0.024)	Loss 1.3878e+00 (1.2533e+00)	Acc@1  66.41 ( 72.66)	Acc@5  82.81 ( 87.56)
Epoch: [88][ 940/1002]	Time  0.324 ( 0.334)	Data  0.000 ( 0.024)	Loss 1.3235e+00 (1.2532e+00)	Acc@1  72.66 ( 72.66)	Acc@5  85.94 ( 87.57)
Epoch: [88][ 950/1002]	Time  0.326 ( 0.334)	Data  0.000 ( 0.024)	Loss 1.0775e+00 (1.2537e+00)	Acc@1  74.22 ( 72.66)	Acc@5  89.84 ( 87.56)
Epoch: [88][ 960/1002]	Time  0.326 ( 0.334)	Data  0.000 ( 0.023)	Loss 1.2018e+00 (1.2535e+00)	Acc@1  73.44 ( 72.66)	Acc@5  85.94 ( 87.57)
Epoch: [88][ 970/1002]	Time  0.324

Epoch: [89][ 170/1002]	Time  0.344 ( 0.338)	Data  0.000 ( 0.026)	Loss 1.3827e+00 (1.2204e+00)	Acc@1  67.97 ( 73.47)	Acc@5  85.16 ( 87.87)
Epoch: [89][ 180/1002]	Time  0.331 ( 0.338)	Data  0.000 ( 0.026)	Loss 1.1199e+00 (1.2202e+00)	Acc@1  75.78 ( 73.47)	Acc@5  91.41 ( 87.85)
Epoch: [89][ 190/1002]	Time  0.336 ( 0.337)	Data  0.000 ( 0.026)	Loss 1.2095e+00 (1.2192e+00)	Acc@1  72.66 ( 73.45)	Acc@5  90.62 ( 87.93)
Epoch: [89][ 200/1002]	Time  0.329 ( 0.337)	Data  0.000 ( 0.026)	Loss 1.3249e+00 (1.2235e+00)	Acc@1  69.53 ( 73.36)	Acc@5  86.72 ( 87.89)
Epoch: [89][ 210/1002]	Time  0.336 ( 0.337)	Data  0.000 ( 0.025)	Loss 1.1049e+00 (1.2240e+00)	Acc@1  72.66 ( 73.32)	Acc@5  86.72 ( 87.90)
Epoch: [89][ 220/1002]	Time  0.333 ( 0.337)	Data  0.000 ( 0.025)	Loss 1.3135e+00 (1.2260e+00)	Acc@1  73.44 ( 73.30)	Acc@5  84.38 ( 87.87)
Epoch: [89][ 230/1002]	Time  0.328 ( 0.337)	Data  0.000 ( 0.025)	Loss 1.2000e+00 (1.2256e+00)	Acc@1  71.88 ( 73.25)	Acc@5  89.84 ( 87.88)
Epoch: [89][ 240/1002]	Time  0.328

Epoch: [89][ 770/1002]	Time  0.326 ( 0.330)	Data  0.000 ( 0.023)	Loss 1.2138e+00 (1.2445e+00)	Acc@1  71.09 ( 72.89)	Acc@5  91.41 ( 87.56)
Epoch: [89][ 780/1002]	Time  0.326 ( 0.330)	Data  0.000 ( 0.023)	Loss 1.2304e+00 (1.2447e+00)	Acc@1  75.00 ( 72.90)	Acc@5  88.28 ( 87.56)
Epoch: [89][ 790/1002]	Time  0.323 ( 0.330)	Data  0.000 ( 0.023)	Loss 1.4180e+00 (1.2444e+00)	Acc@1  67.97 ( 72.91)	Acc@5  85.94 ( 87.57)
Epoch: [89][ 800/1002]	Time  0.324 ( 0.330)	Data  0.000 ( 0.023)	Loss 1.0757e+00 (1.2446e+00)	Acc@1  76.56 ( 72.91)	Acc@5  89.84 ( 87.56)
Epoch: [89][ 810/1002]	Time  0.325 ( 0.330)	Data  0.000 ( 0.023)	Loss 1.2575e+00 (1.2442e+00)	Acc@1  71.88 ( 72.91)	Acc@5  85.94 ( 87.57)
Epoch: [89][ 820/1002]	Time  0.329 ( 0.330)	Data  0.000 ( 0.023)	Loss 1.1339e+00 (1.2442e+00)	Acc@1  73.44 ( 72.90)	Acc@5  89.06 ( 87.57)
Epoch: [89][ 830/1002]	Time  0.325 ( 0.330)	Data  0.000 ( 0.023)	Loss 1.2338e+00 (1.2448e+00)	Acc@1  70.31 ( 72.88)	Acc@5  89.06 ( 87.57)
Epoch: [89][ 840/1002]	Time  0.320

In [5]:
import torchvision.models as torch_models
import torch
from collections import OrderedDict

model = torch_models.resnet18()
    
checkpoint = torch.load('/home/jtang/Desktop/original_model/checkpoint.pth.tar')